In [1]:
from eval_metrics.glossary import *

import json
def load_jsonl(path):
    data=[]
    with open(path, 'r', encoding='utf-8') as reader:
        for line in reader:
            data.append(json.loads(line))
    return data 

ROOT_PATH = "/data/aofei"
dataset = "Slake"
lora_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/lora/epoch3_lung/inference/pred.jsonl"
bv_pred_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/visual_enhance/lung/lora_0.1_bbox_0.05/epoch3/inference/pred.jsonl"

gt_path = f"{ROOT_PATH}/hallucination/{dataset}/data/test_lung.json"

gt = json.load(open(gt_path, 'r'))
# candidate = json.load(open(args.candidate, 'r'))
lora_pred = load_jsonl(lora_pred_path)
bv_pred = load_jsonl(bv_pred_path)

gt_ids = [item['id'] for item in gt]
pred_ids = [item['question_id'] for item in lora_pred]
bv_pred_ids = [item['question_id'] for item in bv_pred]
num_gt_ids, num_pred_ids = len(gt_ids), len(pred_ids)
print(f'num_gt_ids: {num_gt_ids} || num_pred_ids: {num_pred_ids}')
# import pdb; pdb.set_trace()
assert gt_ids == pred_ids, "please make sure pred and gt are exactly matched"
assert gt_ids == bv_pred_ids, "please make sure pred and gt are exactly matched"

num_gt_ids: 419 || num_pred_ids: 419


In [2]:
# the original lora
num_close = 0
correct_ids, wrong_ids = [], []
for gt_item, pred_item in zip(gt, lora_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                correct_ids.append(_id)
            else:
                wrong_ids.append(_id)
        # else:
        #     wrong_ids.append(_id)

In [3]:
len(wrong_ids), len(correct_ids)

(20, 99)

In [4]:
# our method, without MOE
num_close = 0
bv_correct_ids, bv_wrong_ids = [], []
for gt_item, pred_item in zip(gt, bv_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                bv_correct_ids.append(_id)
            else:
                bv_wrong_ids.append(_id)
        # else:
        #     nobbox_wrong_ids.append(_id)

In [5]:
len(bv_wrong_ids), len(bv_correct_ids)

(15, 104)

In [10]:
common_wrong_ids, bv_wrongs, lora_wrongs = [], [], []
for i in wrong_ids:
    if i in bv_wrong_ids:
        common_wrong_ids.append(i)
    else:
        lora_wrongs.append(i)
for i in bv_wrong_ids:
    if i in wrong_ids:
        continue
    else:
        bv_wrongs.append(i)

common_correct_ids, bv_corrects, lora_corrects = [], [], []
for i in correct_ids:
    if i in bv_correct_ids:
        common_correct_ids.append(i)
    else:
        lora_corrects.append(i)
for i in bv_correct_ids:
    if i in correct_ids:
        continue
    else:
        bv_corrects.append(i)
len(lora_corrects), len(bv_corrects)

(4, 9)

In [9]:
gt[0]
gt_id_dict = dict()
for i in gt:
    gt_id_dict[i['id']] = i

In [11]:
lora_corrects
# specific questions correctly answered by LoRA
for i in lora_corrects:
    print(gt_id_dict[i])

{'image': 'xmlab350/source.jpg', 'id': 12408, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab357/source.jpg', 'id': 12423, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab367/source.jpg', 'id': 12453, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the upper right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab85/source.jpg', 'id': 12976, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain lung?'}, {'from': 'gpt', 'value': 'Yes'}]}


In [12]:
bv_corrects
# gt_id_dict[11979]
for i in bv_corrects:
    print(gt_id_dict[i])

{'image': 'xmlab105/source.jpg', 'id': 11961, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab299/source.jpg', 'id': 12273, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab314/source.jpg', 'id': 12333, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab361/source.jpg', 'id': 12438, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan nodule be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab378/source.jpg', 'id': 12496, 'organ': 'L

In [13]:
wrong_after_tuning_ids = list(set(lora_corrects) and set(bv_wrongs))
print(len(wrong_after_tuning_ids), len(lora_corrects), len(bv_wrongs))

correct_after_tuning_ids = list(set(lora_wrongs) and set(bv_corrects))
len(correct_after_tuning_ids), len(lora_wrongs), len(bv_corrects)

4 4 4


(9, 9, 9)

In [14]:
# wrong_after_tuning_ids
for i in wrong_after_tuning_ids:
    print(gt_id_dict[i])

{'image': 'xmlab350/source.jpg', 'id': 12408, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab367/source.jpg', 'id': 12453, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the upper right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab85/source.jpg', 'id': 12976, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain lung?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab357/source.jpg', 'id': 12423, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}


In [15]:
# correct_after_tuning_ids
for i in correct_after_tuning_ids:
    print(gt_id_dict[i])

{'image': 'xmlab314/source.jpg', 'id': 12333, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab378/source.jpg', 'id': 12496, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': "<image>\nIs/Are there abnormalities in the patient's left lung?"}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab299/source.jpg', 'id': 12273, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab378/source.jpg', 'id': 12498, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan infiltration be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab386/source.

In [ ]:
'Can pneumonia be observed on the right lung in this picture?'
"Is/Are there abnormalities in the patient's left lung?"
'Can pneumothorax be observed on the lower left lung in this picture?'
'Can infiltration be observed on the lower left lung in this picture?'
'Can effusion be observed on the lower left lung in this picture?'
'Does the picture contain heart?'
'Does the lung look healthy?'
'Can nodule be observed on the lower right lung in this picture?'
'Does the picture contain heart?'
